# GradConnection

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import re
import json
from datetime import datetime,timedelta
from dateutil.parser import parse
import urllib3

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)


class CustomError(Exception):
    pass

def grad_connection_scrape(job_level,discipline,coverletter_context):
    
    # Set headers to mimic a browser visit
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }

    base_url = f'https://au.gradconnection.com/{job_level}/{discipline}/australia/'
    local_url = 'https://au.gradconnection.com'
    page_num = 1
    jobs_list = []

    while True:
        # Construct the URL for the current page
        if page_num == 1:
            url = f"{base_url}"
        else:
            url = f"{base_url}?page={page_num}"

        # Send a GET request
        response = requests.get(url, headers=headers,verify=False)
        
        # Break the loop if the request failed
        if response.status_code != 200:
            print(f"Failed to retrieve page {page_num}. Status code: {response.status_code}")
            break
        
        # Parse the HTML content
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Find all job listings
        job_listings = soup.find_all('a', class_='box-header-title')

        # Break the loop if no jobs found
        if not job_listings:
            print("No more job listings found.")
            break
        
        print(f"Scraping page {page_num}...")
        
        # Extract details for each job
        index = 0
        for job in job_listings:
            
            job_type = None
            disciplines = None
            work_rights = None
            work_from_home = None
            location = None
            international = None
            closing_date = None
            position_start_date = None
            
            # finds all the secondary link to the job
            job_link = str(job.get('href'))
            if not "notifyme" in job_link:
                current_url = local_url+job_link
                try:
                    response2 = requests.get(current_url, headers=headers,verify = False)
                except CustomError as e:
                    print(f"Custom error caught: {e}")

                # soup2 has all the info about the job
                soup2 = BeautifulSoup(response2.content, 'html.parser')

                # job details contains all info about the job
                job_detail = soup2.find_all('li',class_ = 'box-content-catagories catagories-list')
                for detail in job_detail:
                    #print(detail)
                    strong_tag = detail.find('strong',class_ = 'box-content-catagories-bold').get_text()
                    #print("strong_tag",strong_tag)

                
                    if strong_tag:
                        if strong_tag == "Job type:":
                            job_type = detail.text[9:]
                            #print(job_type)
                        elif strong_tag == "Disciplines:":
                            disciplines = detail.text[11:]
                            #print(disciplines)
                        elif strong_tag == "Work from home:":
                            work_from_home = detail.text[15:]
                            #print(work_from_home)
                        elif strong_tag == "Locations:":
                            location = detail.text[10:]
                            #print(location)
                        elif strong_tag == "ACCEPTS INTERNATIONAL":
                            international = detail.text[21:]
                            #print(international)
                        elif strong_tag == "Closing Date:":
                            closing_date = detail.text[13:]
                            closing_date = parse(closing_date)
                            #print(closing_date)
                        elif strong_tag == "Position Start Date:":
                            position_start_date = detail.text[20:]
                            
                            #print(position_start_date)
                
                    company_name = soup2.find('h1',class_ = 'employers-panel-title').get_text()
                    program_name = soup2.find('h1',class_ = 'employers-profile-h1').get_text()
                    
                # fk pay internship
                if company_name!= "Readygrad" and company_name!= "GradConnection" and company_name!="CareerDC" and company_name!="Premium Graduate Placements":
                    # call the api from ollama
                    #print("company_name: ", company_name)
                    #print("title: ", title)
                    # def remove_think_tags(response):
                    #     # Use regular expression to remove content within <think> tags
                    #     cleaned_response = re.sub(r'<think>.*?</think>', '', response, flags=re.DOTALL)
                    #     return cleaned_response.strip()

                    # api = 'http://localhost:11434/api/generate'
                    # headers = {'Content-Type': 'application/json'}
                    # prompt = f'My name is Richard Shao, I am a {coverletter_context["year_level"]} student majoring in {coverletter_context["major"]} with a minor in {coverletter_context["minor"]}.I have experience in {coverletter_context["ability"]}. One of the relevant event to the company program is {coverletter_context["event"]}. The company I am applying for is {company_name} and the program is {title}. Please write a cover letter for the company {company_name} and the program {title}. Make sure that the cover letter is relevant to the context, and fits the company culture. Please keep the cover letter to one page with is around 500 words. There is no need to mention anything else other than your name on the top of a cover letter. Please double check that the company name is correct. Start the response with: Dear Hiring Manager,and end with Richard Shao'

                    # data = {
                    #     'model': 'deepseek-r1:14b',
                    #     'prompt': prompt,
                    #     'stream': False
                    # }

                    # response = requests.post(api, headers=headers, data=json.dumps(data))
                    # result = remove_think_tags(response.json()["response"])

                    # import os

                    # # Define your target directory and file name
                    # directory = "coverletters_grad"
                    # filename = f"{company_name}_{title}.txt"
                    # # remove illegal character in filename
                    # allowed_fn_chars = [' ','_']
                    # filename = "".join(x for x in filename if (x.isalnum() or x in allowed_fn_chars))

                    # file_path = os.path.join(directory, filename)

                    # # Create the directory if it doesn't exist
                    # os.makedirs(directory, exist_ok=True)
                    
                    # # export result into a txt file named by company name and job title
                    # with open(file_path, 'w') as f:
                    #     f.write(result)
                    
                    jobs_list.append({
                        'Program Title': program_name,
                        'Company': company_name,
                        'Link': current_url,
                        'Job Type': job_type,
                        'Disciplines': disciplines,
                        'Work from Home': work_from_home,
                        'Location': location,
                        'International': international,
                        'Position Start Date': position_start_date,
                        'Closing Date': closing_date
                    })
                    time.sleep(1)
                    print(program_name,company_name)
                    print(f'onto the {index}th job')
                    index+=1
                    # if index == 6:
                    #     time.sleep(60)
    
        # Increment page number and add a delay
        page_num += 1
        time.sleep(2)  # 1-second delay between requests

    # Create DataFrame
    df_jobs = pd.DataFrame(jobs_list)
    df_jobs.sort_values(by='Closing Date', inplace=True)
    df_jobs.to_excel(f'{discipline}_{job_level}_jobs.xlsx', index=False)

    # Display results
    print("\nScraping completed. Found {} relevant jobs.".format(len(df_jobs)))
    return


In [6]:
# please choose from the following job level: graduate-jobs, internships，entry-level-jobs
discipline = "data-science-and-analytics"

# choose your discipline "data-science-and-analytics", "computer-science"
job_level = 'internships'

# cover letter context, year_level, major, minor, interest,company_name, job_title company_description
ability = "different programing languages including R,python,java,C,SQL. Differnet packages include numpy, pandas, matplotlib, ggplot2,pytorch,scikit-learn,spark"
event = "help processing data from various news sources, and help to build a model to extract emotion and company name from the news as well as putting a label to the news "
coverletter_context = {"year_level":"first year master","major":"Probability and Statisitcs",
                       "minor":"Data Science","company_name":"",
                       "job_program":"","ability":ability,"event":event}

grad_connection_scrape(job_level,discipline,coverletter_context)


Scraping page 1...
EY 2025/26 Vacationer Data Analytics Program EY
onto the 0th job
Quantitative Researcher Internship - 2025-26 Optiver
onto the 1th job
Quantitative Trading Internship - 2025/26 Optiver
onto the 2th job
EY 2025/26 Vacationer Information Security Program EY
onto the 3th job
2025 Winter Development Program (Perth) Macquarie Group
onto the 4th job
2026 KPMG Vacationer Program – Risk KPMG
onto the 5th job
2026 KPMG Vacationer Program – Technology and Digital KPMG
onto the 6th job
2026 KPMG Vacationer Program – Engineering KPMG
onto the 7th job
EY 2025/26 Vacationer Information Technology / Information Systems / Business Information Systems Program EY
onto the 8th job
2025 Winter Development Program (Melbourne) Macquarie Group
onto the 9th job
2025 Winter Development Program (Sydney) Macquarie Group
onto the 10th job
2026 KPMG Vacationer Program – Business and Consulting KPMG
onto the 11th job
2025-2026 NAB Summer Intern Program - Expression of Interest NAB
onto the 12th j

# Seek

https://www.seek.com.au/internship-machine-learning-jobs/in-All-Australia?classification=1223%2C6281

In [54]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import re

# please choose from the following job level: graduate-jobs, internships，entry-level-jobs
seek_keyword = f'{job_level}-machine-learning'

# Set headers to mimic a browser visit
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

# Initialize list to store job data and keywords for filtering
jobs_list = []


# choose your discipline (see seek website for details)
discipline = "1223%2C6281" # All ICT and Science&Technology
word_discipline = "science_and_technology"
base_url = f'https://www.seek.com.au/{seek_keyword}-jobs/in-All-Australia?classification={discipline}'
local_url = 'https://www.seek.com.au'
page_num = 1

while True:
    # Construct the URL for the current page
    if page_num == 1:
        url = f"{base_url}"
    else:
        url = f"{base_url}&page={page_num}"

    # Send a GET request
    print(f"Scraping page {page_num}...")
    response = requests.get(url, headers=headers)
    
    # Break the loop if the request failed
    if response.status_code != 200:
        print(f"Failed to retrieve page {page_num}. Status code: {response.status_code}")
        break
    
    # Parse the HTML content
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Find all job listings
    job_listings = soup.find_all('article')
    # Break the loop if no jobs found
    if not job_listings:
        print("No more job listings found.")
        break
    
    print(f"Analysing page {page_num}...")
    
    # Extract details for each job
    for job in job_listings:

        title = job.get('aria-label')
        url_extra = job.find('a')['href']

        url_new = local_url+url_extra
        
        company = job.find('a', {'data-type': 'company'})
        if company:
            company_name = company.text
        else:
            company_name = None

        if company_name:
            # company_name = company_name.group(1)
                # Add to jobs list
               
            # fk pay internship
            if company_name!= "readygrad" and company_name!= "gradconnection":
                def remove_think_tags(response):
                        # Use regular expression to remove content within <think> tags
                        cleaned_response = re.sub(r'<think>.*?</think>', '', response, flags=re.DOTALL)
                        return cleaned_response.strip()

                api = 'http://localhost:11434/api/generate'
                headers = {'Content-Type': 'application/json'}
                prompt = f'My name is Richard Shao, I am a {coverletter_context["year_level"]} student majoring in {coverletter_context["major"]} with a minor in {coverletter_context["minor"]}.I have experience in {coverletter_context["ability"]}. One of the relevant event to the company program is {coverletter_context["event"]}. The company I am applying for is {company_name} and the program is {title}. Please write a cover letter for the company {company_name} and the program {title}. Make sure that the cover letter is relevant to the context, and fits the company culture. Please keep the cover letter to one page with is around 500 words. There is no need to mention anything else other than your name on the top of a cover letter. Please double check that the company name is correct. Start the response with: Dear Hiring Manager,and end with Richard Shao'

                data = {
                    'model': 'deepseek-r1:14b',
                    'prompt': prompt,
                    'stream': False
                }

                response = requests.post(api, headers=headers, data=json.dumps(data))
                result = remove_think_tags(response.json()["response"])

                import os

                # Define your target directory and file name
                directory = "coverletters_seek"
                filename = f"{company_name}_{title}.txt"
                # remove illegal character in filename
                allowed_fn_chars = [' ','_']
                filename = "".join(x for x in filename if (x.isalnum() or x in allowed_fn_chars))

                file_path = os.path.join(directory, filename)

                # Create the directory if it doesn't exist
                os.makedirs(directory, exist_ok=True)
                
                # export result into a txt file named by company name and job title
                with open(file_path, 'w') as f:
                    f.write(result)
                jobs_list.append({
                    'Program Title': title,
                    'Company': company_name,
                    'Link': url_new,
                })



    
    # Increment page number and add a delay
    page_num += 1
    time.sleep(5)  # 1-second delay between requests


# append into df_jobs
df_jobs = pd.DataFrame(jobs_list)


# Display results
print("\nScraping completed. Found {} relevant jobs.".format(len(df_jobs)))
print("\nSample of scraped jobs:")



#To save to CSV:
df_jobs.to_excel(f'{word_discipline}_{job_level}.xlsx', index=False)

Scraping page 1...
Analysing page 1...
Scraping page 2...
Analysing page 2...
Scraping page 3...
Analysing page 3...


KeyboardInterrupt: 

In [7]:
# change datascience_graduate-jobs to excel
df_jobs.to_excel(f'datascience_{job_level}.xlsx', index=False)